In [11]:
import numpy as np
import pandas as pd
import tweepy
import json
from tweepy import OAuthHandler

from textblob import TextBlob
import re

In [3]:
consumer_key = "Prh8zd6Jp4MtVds2fK3WqTR2R"
consumer_secret = "GUmsef8iPNkVVNS6YrzKViOXpdgLqWKVSwKztFVV5SG6fI9gQA"
access_token = "1321731436458639360-B4DSVovNO9hEyILb2G18OpB0QEfT7C"
access_token_secret = "tMLCP0tg12c12guwNHN0Sf16jvVGMtKDhytRJOlpB72Pw"

In [4]:
auth = tweepy.OAuthHandler( consumer_key , consumer_secret )
auth.set_access_token( access_token , access_token_secret )
api = tweepy.API(auth)

In [5]:
query = "Sushant Singh Rajput"

In [6]:
Tweets = api.search( query , count=10 , lang='en' , exclude='retweets' , tweet_mode='extended' )

for tweet in Tweets:
    print(tweet,"\n")
    print(".....................................\n")

Status(_api=<tweepy.api.API object at 0x00000203F39A9430>, _json={'created_at': 'Thu Oct 29 10:20:39 +0000 2020', 'id': 1321758839570526210, 'id_str': '1321758839570526210', 'full_text': "@narendramodi \nWe trust you!\nWe respect you!\nWe follow you!\nPpl all around the world are now looking up to u sir. Justice for Sushant Singh Rajput is global demand.Plz sir kindly looking into the matter,don't disappoint us🙏😓 #Justice4SSR https://t.co/XvmNiK7Yj0", 'truncated': False, 'display_text_range': [0, 237], 'entities': {'hashtags': [{'text': 'Justice4SSR', 'indices': [225, 237]}], 'symbols': [], 'user_mentions': [{'screen_name': 'narendramodi', 'name': 'Narendra Modi', 'id': 18839785, 'id_str': '18839785', 'indices': [0, 13]}], 'urls': [], 'media': [{'id': 1321758781282246656, 'id_str': '1321758781282246656', 'indices': [238, 261], 'media_url': 'http://pbs.twimg.com/media/ElfUm80U4AAOS1m.jpg', 'media_url_https': 'https://pbs.twimg.com/media/ElfUm80U4AAOS1m.jpg', 'url': 'https://t.co/XvmNiK7

In [7]:
df = pd.DataFrame(columns = ['Tweets' , 'User' , 'User_statuses_count' , 
                            'user_followers' , 'User_location' , 'User_verified' ,
                            'fav_count' , 'rt_count' , 'tweet_date'] )
print(df)

Empty DataFrame
Columns: [Tweets, User, User_statuses_count, user_followers, User_location, User_verified, fav_count, rt_count, tweet_date]
Index: []


In [8]:
df.shape

(0, 9)

In [9]:
def stream(data, file_name):
    i = 0
    for tweet in tweepy.Cursor(api.search, q=data, count=100, lang='en').items():
        print(i, end='\r')
        df.loc[i, 'Tweets'] = tweet.text
        df.loc[i, 'User'] = tweet.user.name
        df.loc[i, 'User_statuses_count'] = tweet.user.statuses_count
        df.loc[i, 'user_followers'] = tweet.user.followers_count
        df.loc[i, 'User_location'] = tweet.user.location
        df.loc[i, 'User_verified'] = tweet.user.verified
        df.loc[i, 'fav_count'] = tweet.favorite_count
        df.loc[i, 'rt_count'] = tweet.retweet_count
        df.loc[i, 'tweet_date'] = tweet.created_at
        df.to_excel('{}.xlsx'.format(file_name))
        i = i+1
        if i == 1000:
            break
        else:
            pass

In [10]:
stream(data=[query] , file_name='my_tweets')

In [12]:
def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity == 0:
        return 'Neutral'
    else:
        return 'Negative'

In [13]:
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', tweet).split())

In [14]:
df['clean_tweet'] = df['Tweets'].apply(lambda x : clean_tweet(x))

In [15]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet
0,RT @RadhaMurthy3: WE MAY HAVE DIFFERENCE OF OP...,Shri 🕊,23425,245,,False,0,8,2020-10-29 10:23:08,RT WE MAY HAVE DIFFERENCE OF OPINIONS BUT WE W...
1,RT @RiyushaA95: All Love and addiction only fo...,Jyoti Rathod,7605,368,,False,0,225,2020-10-29 10:23:00,RT All Love and addiction only for Sushant Sin...
2,RT @ItsHarshi07: I can't be described in words...,Ruhi,41768,979,Twitter,False,0,2,2020-10-29 10:22:58,RT I can t be described in words Mansoor was d...
3,RT @arnabofficial9: It's High time now to impo...,Himashree Popat,55,2,,False,0,605,2020-10-29 10:22:53,RT It s High time now to impose IPC section 30...
4,RT @saketjaiswal_sj: Today Justice for Sushant...,Nilima ....... n I m not a boat,5914,36,,False,0,430,2020-10-29 10:22:49,RT sj Today Justice for Sushant Singh Rajput F...


In [16]:
df['Sentiment'] = df['clean_tweet'].apply(lambda x : analyze_sentiment(x) )
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet,Sentiment
0,RT @RadhaMurthy3: WE MAY HAVE DIFFERENCE OF OP...,Shri 🕊,23425,245,,False,0,8,2020-10-29 10:23:08,RT WE MAY HAVE DIFFERENCE OF OPINIONS BUT WE W...,Neutral
1,RT @RiyushaA95: All Love and addiction only fo...,Jyoti Rathod,7605,368,,False,0,225,2020-10-29 10:23:00,RT All Love and addiction only for Sushant Sin...,Positive
2,RT @ItsHarshi07: I can't be described in words...,Ruhi,41768,979,Twitter,False,0,2,2020-10-29 10:22:58,RT I can t be described in words Mansoor was d...,Positive
3,RT @arnabofficial9: It's High time now to impo...,Himashree Popat,55,2,,False,0,605,2020-10-29 10:22:53,RT It s High time now to impose IPC section 30...,Positive
4,RT @saketjaiswal_sj: Today Justice for Sushant...,Nilima ....... n I m not a boat,5914,36,,False,0,430,2020-10-29 10:22:49,RT sj Today Justice for Sushant Singh Rajput F...,Neutral


In [17]:
n = 12
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 RT @Shreya4SSR: We may have different opinions, 
but we will fighting for 'Justice for Sushant Singh Rajput' together.
Unity is our Strengt…

Clean tweet:
 RT We may have different opinions but we will fighting for Justice for Sushant Singh Rajput together Unity is our Strengt

Sentiment of the tweet:
 Neutral


In [18]:
n = 20
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 RT @iRaviTiwari: 134 days have been passed.

WHO KILLED SUSHANT SINGH RAJPUT?

WHO KILLED DISHA SALIAN?
Why Not Arrest Culprits ???
#PMGive…

Clean tweet:
 RT 134 days have been passed WHO KILLED SUSHANT SINGH RAJPUT WHO KILLED DISHA SALIAN Why Not Arrest Culprits PMGive

Sentiment of the tweet:
 Negative


In [28]:
n = 1
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 RT @RiyushaA95: All Love and addiction only for Sushant Singh Rajput... 

#IndiaFightsBack4SSR 
#NationWithFearlessArnab https://t.co/aBlPP…

Clean tweet:
 RT All Love and addiction only for Sushant Singh Rajput IndiaFightsBack4SSR NationWithFearlessArnab

Sentiment of the tweet:
 Positive
